<a href="https://colab.research.google.com/github/Anson422/Anson-Eva-Valerie/blob/main/Text%20Analysis/Code_Text_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/Anson422/Anson-Eva-Valerie.git #clone the repository to the local machine
!pip install PyPDF2

Cloning into 'Anson-Eva-Valerie'...
remote: Enumerating objects: 1034, done.
remote: Counting objects: 100% (271/271), done.
remote: Compressing objects: 100% (243/243), done.
remote: Total 1034 (delta 135), reused 27 (delta 27), pack-reused 763 (from 2)
Receiving objects: 100% (1034/1034), 1.42 GiB | 28.34 MiB/s, done.
Resolving deltas: 100% (362/362), done.
Updating files: 100% (526/526), done.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 3.6 MB/s eta 0:00:00


In [2]:
#install the necessary libraries
import os
import PyPDF2
from collections import Counter
import pandas as pd

In [11]:
#Checking if the cloning was successfull
# Navigate to the cloned repository folder
repository_path = '/content/Anson-Eva-Valerie'

if os.path.exists(repository_path):
    print("Repository cloned successfully!")
    print("Files in the repository:", os.listdir(repository_path))
else:
    print("Repository cloning failed.")

Repository cloned successfully!
Files in the repository: ['Text Analysis', 'hello.ipynb', '.git', 'Stacked Bar Chart', 'Choropleth Map Analysis', 'README.md']


In [19]:
# Define the folder containing the EU energy publications in 2024
EU_Energy_publications_2024 = '/content/Anson-Eva-Valerie/Text Analysis/EU publications about Energy in 2024'

In [21]:
# List of keywords to analyze
keywords = ["LNG", "renewables", "pipeline alternatives", "coal", "oil", "natural gas", "nuclear", "hydro", "biofuels", "geothermal", "solar", "wind", "wave power", "biogas", "biomass", "water"]

# Initialize a counter for keyword frequency
keyword_counts = Counter()

In [20]:
import os

# List files and folders in the current directory
print(os.listdir('/content'))

['.config', 'Anson-Eva-Valerie', 'sample_data']


In [ ]:
# Function to extract text from a PDF
def extract_text_from_pdf(pdf_path):
    text = ""
    with open(pdf_path, "rb") as file:
        reader = PyPDF2.PdfReader(file)
        for page in reader.pages:
            text += page.extract_text()
    return text

# Loop through all PDFs in the folder
for filename in os.listdir(EU_Energy_publications_2024):
    if filename.endswith(".pdf"):
        pdf_path = os.path.join(EU_Energy_publications_2024, filename)
        print(f"Processing: {filename}")

        # Extract text and count keywords
        text = extract_text_from_pdf(pdf_path).lower()
        for keyword in keywords:
            keyword_counts[keyword] += text.count(keyword.lower())

# Convert results to a DataFrame for better visualization
df = pd.DataFrame(keyword_counts.items(), columns=["Keyword", "Frequency"])
print(df)

Processing: resilience%20assessment-KJ0124002ENN.pdf
Processing: behavioural%20insights%20applied%20to%20policy-KJ0124189ENN.pdf
Processing: addressing%20the%20regional%20dimension%20of%20open%20strategic-KJNA31868ENN.pdf
Processing: strukturlotsen%20improving%20inter-municipal%20cooperation-MJ0524516ENN.pdf
Processing: europeans%20opinions%20about%20the%20european%20unions%20priorities-NAAO24004ENN.pdf
Processing: europeans%20attitudes%20towards%20eu%20energy%20policy-MJ0924678ENN.pdf
Processing: bulgaria%20and%20its%20collaboration%20with%20the%20joint%20research-KJ0624149ENN.pdf
Processing: fisa%202022-KI0323060ENN-compressed.pdf
Processing: clean%20energy%20technology%20observatory%20hydropower%20and-KJ0124070ENN.pdf
Processing: clean%20energy%20technology%20observatory%20early%20stage%20technologies-KJNA31807ENN.pdf
Processing: ramp-up%20and%20role%20of%20hydrogen-based%20power%20generation-MJ0524460ENN.pdf
Processing: smart%20cities%20marketplace%20newsletter%2030%20may%202024-MJ

[0, IndirectObject(42, 0, 138028866111088)]


Processing: eic%20pathfinder%20portfolio-EA0524617ENN.pdf
Processing: making%20it%20happen-EG0924481ENN.pdf
Processing: towards%20a%20climate%20and%20energy%20union-QA0124055ENN.pdf
Processing: regional%20profile-MJ0224839ENN.pdf
Processing: local%20clean%20energy%20transition-ZZ0624047ENN.pdf
Processing: a%20positive%20trade-off-QH0124014ENN.pdf
Processing: annual%20impact%20report-QG0224146ENN.pdf
Processing: vocational%20and%20educational%20initiatives%20to%20re-%20and-KN0524494ENN.pdf
Processing: monitoring%20the%20twin%20transition%20of%20industrial%20ecosystems-EA0523534ENN.pdf
Processing: clean%20hydrogen%20joint%20undertaking%20and%20its%20predecessor-KI0924618ENN.pdf
Processing: towards%20a%20climate-neutral%20and%20climate-resilient%20eu-TH0224650ENN.pdf
Processing: cordis%20results%20pack%20on%20in-space%20electric%20propulsion-ZZ0924228ENN.pdf
Processing: assessment%20of%20the%20nuclear%20risk%20in%20europe-KJNA31038ENN.pdf
Processing: development%20of%20outlook%20for%20the

[0, IndirectObject(3335, 0, 138028876995296)]


Processing: partnership%20development%20and%20governance%20for%20a%20just-MJ0524522ENN.pdf
Processing: contracts-for-difference%20to%20support%20renewable%20energy-QM0224345ENN.pdf
Processing: citizen-led%20renovation%203%202024-MJBC24003ENN.pdf
Processing: regional%20profile-MJ0524524ENN.pdf
Processing: assessing%20the%20impact%20of%20energy%20efficiency%20on%20the%20eu-KJNA31971ENN.pdf
Processing: just%20transition%20platform%20policy%20approach-KN0924414ENN.pdf
Processing: decarbonisation%20of%20residential%20heating%20and%20cooling-TJ0124005ENN.pdf
Processing: in-depth%20review%202024-KCBC24010ENN.pdf
Processing: the%20persistence%20of%20energy%20poverty%20in%20the%20eu-KJNA32008ENN.pdf
Processing: impact%20assessment%20on%20the%20proposal%20for%20a%20network%20code-MJ0224572ENN.pdf
Processing: future%20usesneeds%20of%20the%20seas-HZ0124013ENN.pdf
Processing: international%20cooperation%20on%20carbon%20pricing-QM0124132ENN.pdf
Processing: the%20future%20of%20solar-to-x-EA0124045ENN